In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 17.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=6dbba4512ddc70fd1204329f7f8b1fb7afed8b481c635e5dbc0d3e8900f1e40d
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import length
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.ml.feature import CountVectorizer, IDF, StringIndexer
from pyspark.ml.classification import NaiveBayes, RandomForestClassifier, LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector
from pyspark.ml import Pipeline
from pyspark.sql.functions import lit
from bs4 import BeautifulSoup
from pyspark import keyword_only
import pyspark.sql.functions as F
from pyspark.mllib.evaluation import MulticlassMetrics
from datetime import datetime
from pyspark.ml import Transformer
from pyspark.sql.types import FloatType
from pyspark.ml.param.shared import HasInputCol, HasOutputCol 
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType 
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

In [ ]:
spark = SparkSession.builder.appName('Rating').getOrCreate()

### Load Data

In [ ]:
df1 = spark.read.csv("/content/drive/MyDrive/comments_final.csv",header=True
                      ,inferSchema=True)

In [ ]:
#df1 = df1.withColumn('class',lit('fake'))

In [ ]:
df1.show(2)

+---+--------------------+--------+
|_c0|      processed_text|   class|
+---+--------------------+--------+
|  0|    rất_hài sản_phẩm|positive|
|  1|sản_phẩm chất_lượ...|positive|
+---+--------------------+--------+
only showing top 2 rows



In [ ]:
df1.count()

134231

In [ ]:
df1.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- processed_text: string (nullable = true)
 |-- class: string (nullable = true)



In [ ]:
df1.groupby('class').count().show()

+--------+------+
|   class| count|
+--------+------+
|positive|103594|
| neutral|  9842|
|negative| 20795|
+--------+------+



### Precprocess

In [ ]:
df1 = df1.withColumn('length', length(df1['processed_text']))

In [ ]:
df1.show(5)

+---+--------------------+--------+------+
|_c0|      processed_text|   class|length|
+---+--------------------+--------+------+
|  0|    rất_hài sản_phẩm|positive|    16|
|  1|sản_phẩm chất_lượ...|positive|    85|
|  2|sản_phẩm chất_lượ...|positive|    85|
|  3|cuộc_sống không_g...|positive|    70|
|  4|viết nhận_xét quá...|negative|   543|
+---+--------------------+--------+------+
only showing top 5 rows



In [ ]:
df1.groupBy('class').mean().show()

+--------+-----------------+------------------+
|   class|         avg(_c0)|       avg(length)|
+--------+-----------------+------------------+
|positive| 65855.4977797942| 46.68221132498021|
| neutral|68782.34423897581|46.808169071326965|
|negative|72600.30276508776|51.288627073815825|
+--------+-----------------+------------------+



In [ ]:
data = df1.select('processed_text','class','length')

In [ ]:
data.show()

+--------------------+--------+------+
|      processed_text|   class|length|
+--------------------+--------+------+
|    rất_hài sản_phẩm|positive|    16|
|sản_phẩm chất_lượ...|positive|    85|
|sản_phẩm chất_lượ...|positive|    85|
|cuộc_sống không_g...|positive|    70|
|viết nhận_xét quá...|negative|   543|
|hàng bị_lỗi màng ...|negative|    69|
|sản_phẩm rất_tốt ...|positive|    57|
|rất_hài sản_phẩm ...|positive|    23|
|băng xài ổn lực c...|positive|    32|
|sản_phẩm sản_phẩm...| neutral|    90|
|hàng băng bị_cắt sài|negative|    20|
|mẫu_mã đẹp chất_l...|positive|    21|
|sản_phẩm không_đú...|negative|    30|
|hàng số_lượng đơn...|negative|    47|
|không_nói đừng sả...|negative|    23|
|dán vô rớt hoài v...|negative|    32|
|                băng|positive|     4|
|hàng đúng_hạn đón...|positive|    35|
|gọn bỏ_túi quảng_...|positive|    71|
|như_hình dùm chất...|positive|    26|
+--------------------+--------+------+
only showing top 20 rows



In [ ]:
null_data = data.filter(data['processed_text'].isNull())

In [ ]:
null_data.count()

0

In [ ]:
data = data.filter(data['processed_text'].isNotNull())

### Feature & Transform

In [ ]:
class BsTextExtractor(Transformer, HasInputCol, HasOutputCol):
    
    @keyword_only
    def __init__(self, inputCol=None, outputCol=None):
        super(BsTextExtractor, self).__init__() 
        kwargs = self._input_kwargs 
        self.setParams(**kwargs)
        
    @keyword_only
    def setParams(self, inputCol=None, outputCol=None): 
        kwargs = self._input_kwargs
        return self._set(**kwargs)
    def _transform(self, dataset):
        def f(s):
            cleaned_text = BeautifulSoup(s).text 
            return cleaned_text
        t = StringType()
        out_col = self.getOutputCol()
        in_col = dataset[self.getInputCol()]
        return dataset.withColumn(out_col, udf(f, t)(in_col))

In [ ]:
text_extractor = BsTextExtractor(inputCol="processed_text", outputCol="cleaned_text")
tokenizer = Tokenizer(inputCol='cleaned_text', outputCol='token_text')
stopremove= StopWordsRemover(inputCol='token_text', outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label', handleInvalid='keep')

In [ ]:
clean_up = VectorAssembler(inputCols =['tf_idf','length'],
                           outputCol='features')

AttributeError: ignored

In [ ]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,
                                  text_extractor,
                                  tokenizer,
                                  stopremove,
                                  count_vec,
                                  idf,
                                  clean_up])

In [ ]:
cleaner = data_prep_pipe.fit(data)

In [ ]:
clean_data = cleaner.transform(data)

In [ ]:
clean_data = clean_data.select('label','features')

In [ ]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(27964,[1,53,2796...|
|  0.0|(27964,[0,1,5,8,2...|
|  0.0|(27964,[0,1,5,8,2...|
|  0.0|(27964,[428,784,1...|
|  1.0|(27964,[55,105,33...|
|  1.0|(27964,[0,17,23,1...|
|  0.0|(27964,[0,1,48,59...|
|  0.0|(27964,[1,53,369,...|
|  0.0|(27964,[9,11,234,...|
|  2.0|(27964,[1,3,5,10,...|
|  1.0|(27964,[0,80,239,...|
|  0.0|(27964,[4,5,174,2...|
|  1.0|(27964,[1,5,169,2...|
|  1.0|(27964,[0,67,330,...|
|  1.0|(27964,[1,336,923...|
|  1.0|(27964,[59,263,44...|
|  0.0|(27964,[239,27963...|
|  0.0|(27964,[0,8,80,12...|
|  0.0|(27964,[3,22,50,9...|
|  0.0|(27964,[9,89,243,...|
+-----+--------------------+
only showing top 20 rows



In [ ]:
clean_data.groupBy('label').count().show()

+-----+------+
|label| count|
+-----+------+
|  0.0|103594|
|  1.0| 20795|
|  2.0|  9842|
+-----+------+



In [ ]:
(train, test) = clean_data.randomSplit([0.7,0.3])

## Buil Model

### Naive Bayes

In [ ]:
nb = NaiveBayes()
prediction = nb.fit(train)
test_results = prediction.transform(test)
start_time = datetime.now()
train_time = datetime.now() - start_time  
test_results.show()
print(train_time)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(27964,[0,1,2,3,4...|[-11240.538702246...|[1.0,5.5282785272...|       0.0|
|  0.0|(27964,[0,1,2,3,4...|[-784.25589375873...|[1.62013689002517...|       2.0|
|  0.0|(27964,[0,1,2,3,4...|[-378.53524001305...|[0.99999999999995...|       0.0|
|  0.0|(27964,[0,1,2,3,4...|[-353.10947965994...|[1.0,4.2265688015...|       0.0|
|  0.0|(27964,[0,1,2,3,4...|[-212.17752488656...|[0.99999999071903...|       0.0|
|  0.0|(27964,[0,1,2,3,4...|[-1163.1577445122...|[3.14420084955976...|       2.0|
|  0.0|(27964,[0,1,2,3,4...|[-892.74871919934...|[1.0,8.2561252708...|       0.0|
|  0.0|(27964,[0,1,2,3,4...|[-292.46038644260...|[1.0,3.4656755348...|       0.0|
|  0.0|(27964,[0,1,2,3,4...|[-922.09841256210...|[1.0,8.5043860146...|       0.0|
|  0.0|(27964,[0

In [ ]:
from pyspark.sql.functions import col, explode, array, lit
major_df = data.filter(col("class") == 'positive')
minor_df = data.filter(col("class") == 'negative')
normal_df = data.filter(col("class") == 'neutral')
ratio = int(major_df.count()/minor_df.count()/normal_df.count())
print("ratio: {}".format(ratio))

In [ ]:
test_results.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  2.0|       0.0|  930|
|  1.0|       1.0| 4075|
|  0.0|       1.0| 2382|
|  1.0|       0.0|  754|
|  2.0|       2.0| 1073|
|  2.0|       1.0| 1005|
|  1.0|       2.0| 1376|
|  0.0|       0.0|24902|
|  0.0|       2.0| 3776|
+-----+----------+-----+



### Report

In [ ]:
#important: need to cast to float type, and order by prediction, else it won't work
preds_and_labels = test_results.select(['prediction','label']).withColumn('label', F.col('label').cast(FloatType())).orderBy('prediction')

#select only prediction and label columns
preds_and_labels = preds_and_labels.select(['prediction','label'])

metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))
print(metrics.confusionMatrix().toArray())

/usr/local/lib/python3.9/dist-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


[[24902.  2382.  3776.]
 [  754.  4075.  1376.]
 [  930.  1005.  1073.]]


In [ ]:
acc_eva_nb = MulticlassClassificationEvaluator()
acc_nb = acc_eva_nb.evaluate(test_results)
print('Accuracy of model: {}'.format(acc_nb))

Accuracy of model: 0.77555767485527


### Logistic Regression

In [ ]:
lg = LogisticRegression(maxIter=10, regParam=0.3)
pre_lg = lg.fit(train)
result_lg = pre_lg.transform(test)
start_time = datetime.now()
train_time = datetime.now() - start_time  
print(train_time)
result_lg.groupBy('label', 'prediction').count().show()

0:00:00.000060
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  2.0|       0.0| 2657|
|  1.0|       1.0| 2093|
|  0.0|       1.0|  325|
|  1.0|       0.0| 4061|
|  2.0|       2.0|   24|
|  2.0|       1.0|  327|
|  1.0|       2.0|   51|
|  0.0|       0.0|30687|
|  0.0|       2.0|   48|
+-----+----------+-----+



### Report

In [ ]:
#important: need to cast to float type, and order by prediction, else it won't work
preds_and_labels_lg = result_lg.select(['prediction','label']).withColumn('label', F.col('label').cast(FloatType())).orderBy('prediction')

#select only prediction and label columns
preds_and_labels_lg = preds_and_labels_lg.select(['prediction','label'])

metrics_lg = MulticlassMetrics(preds_and_labels_lg.rdd.map(tuple))
print(metrics_lg.confusionMatrix().toArray())

/usr/local/lib/python3.9/dist-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


[[3.0687e+04 3.2500e+02 4.8000e+01]
 [4.0610e+03 2.0930e+03 5.1000e+01]
 [2.6570e+03 3.2700e+02 2.4000e+01]]


In [ ]:
acc_eva_lg = MulticlassClassificationEvaluator()
acc_lg = acc_eva_lg.evaluate(result_lg)
print('Accuracy of model: {}'.format(acc_lg))

Accuracy of model: 0.7645651255262982
